In [ ]:
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import scipy.ndimage as ndimage

In [ ]:
array = np.load(Path("/Users/sylvi/topo_data/hariborings/testing_curvature/curvature_splined_trace.npy"))

plt.scatter(array[:, 0], array[:, 1])
plt.show()

# Convert the spline to a pixelated trace 1 pixel thick
# Create a map of pixels

min_x = np.min(array[:, 0])
max_x = np.max(array[:, 0])
min_y = np.min(array[:, 1])
max_y = np.max(array[:, 1])
pixel_map = np.zeros((int(np.max(array) + 1), int(np.max(array) + 1)), dtype=int)

plt.plot(array[:, 0], array[:, 1])
plt.show()

plt.imshow(pixel_map)


def check_is_touching(coordinate, original_coordinate):
    if np.abs(coordinate[0] - original_coordinate[0]) <= 1 and np.abs(coordinate[1] - original_coordinate[1]) <= 1:
        return True
    else:
        return False


# Convert the array to integers and remove duplicates
integer_array = np.array(array, dtype=int)
removed_duplicates = []
for index in range(len(integer_array)):
    coordinate = integer_array[index]
    if index > 0:
        if np.array_equal(coordinate, integer_array[index - 1]):
            # print(f"coordinate {coordinate} is a repeat of {integer_array[index - 1]}, skipping")
            continue

    removed_duplicates.append(coordinate)
integer_array = np.array(removed_duplicates)
print(f"integer array: {integer_array}")


# print(f"integer array: {integer_array}")

# Create an image out of the integer array
integer_array_image = np.zeros((np.max(integer_array[:, 0] + 1), np.max(integer_array[:, 1] + 1)))
for coordinate in integer_array:
    integer_array_image[int(coordinate[0]), int(coordinate[1])] = 1

plt.imshow(integer_array_image)
plt.title("Integer array image")
plt.show()


last_coordinate = None
for index in range(len(integer_array)):
    # if index > 20:
    # break

    coordinate = integer_array[index]

    # If the coordinate is a repeat, skip it
    if index > 0:
        if np.array_equal(coordinate, integer_array[index - 1]):
            print(f"coordinate {coordinate} is a repeat of {integer_array[index - 1]}, skipping")
            continue

    print(f"coordinate: {coordinate}")
    if index == 0:
        pixel_map[coordinate[0], coordinate[1]] = 1
        last_coordinate = coordinate
    elif index == len(integer_array) - 1:
        pixel_map[coordinate[0], coordinate[1]] = 1
        last_coordinate = coordinate
        break
    else:
        # Check if the coordinate after this one is touching the coordinate before this one
        # and if so, skip this pixel
        if check_is_touching(integer_array[index + 1], last_coordinate):
            print(f"coordinate {integer_array[index + 1]} is touching {last_coordinate}")
            continue
        else:
            print(f"coordinate {integer_array[index+1]} is not touching {integer_array[index - 1]}. Adding to map")
            pixel_map[int(coordinate[0]), int(coordinate[1])] = 1
            last_coordinate = coordinate

    # plt.imshow(pixel_map)
    # plt.show()

plt.imshow(pixel_map)

In [ ]:
def turn_path_into_pixel_map(array: np.ndarray):
    # Convert the spline to a pixelated trace 1 pixel thick

    # Create a map of pixels
    pixel_map = np.zeros((int(np.max(array) + 1), int(np.max(array) + 1)), dtype=int)
    pixelated_path = np.empty((0, 2), dtype=int)

    def check_is_touching(coordinate, original_coordinate):
        if np.abs(coordinate[0] - original_coordinate[0]) <= 1 and np.abs(coordinate[1] - original_coordinate[1]) <= 1:
            return True
        else:
            return False

    # Convert the array to integers and remove duplicates
    integer_array = np.array(array, dtype=int)
    removed_duplicates = []
    for index in range(len(integer_array)):
        coordinate = integer_array[index]
        if index > 0:
            if np.array_equal(coordinate, integer_array[index - 1]):
                continue

        removed_duplicates.append(coordinate)
    integer_array = np.array(removed_duplicates)

    last_coordinate = None
    for index in range(len(integer_array)):
        coordinate = integer_array[index]

        # If the coordinate is a repeat, skip it
        if index > 0:
            if np.array_equal(coordinate, integer_array[index - 1]):
                continue
        if index == 0:
            pixel_map[coordinate[0], coordinate[1]] = 1
            last_coordinate = coordinate
        elif index == len(integer_array) - 1:
            pixel_map[coordinate[0], coordinate[1]] = 1
            last_coordinate = coordinate
            break
        else:
            # Check if the coordinate after this one is touching the coordinate before this one
            # and if so, skip this pixel
            if check_is_touching(integer_array[index + 1], last_coordinate):
                continue
            else:
                # Add the coordinate to the pixel map and the pixelated path
                pixel_map[int(coordinate[0]), int(coordinate[1])] = 1
                pixelated_path = np.vstack((pixelated_path, coordinate.reshape(1, 2)))
                last_coordinate = coordinate

    return pixel_map, pixelated_path

In [ ]:
# Test it

array = np.load(Path("/Users/sylvi/topo_data/hariborings/testing_curvature/curvature_splined_trace.npy"))

pixel_map, pixelated_path = turn_path_into_pixel_map(array)

plt.imshow(pixel_map)
plt.show()

print(f"pixelated path: {pixelated_path}")

print(type(pixelated_path))
print(pixelated_path.shape)

plt.scatter(pixelated_path[:, 0], pixelated_path[:, 1])

plt.show()

plt.plot(pixelated_path[:, 0], pixelated_path[:, 1])
plt.show()